In [102]:
%glue_version 4.0
%number_of_workers 2
%idle_timeout 10
%worker_type G.2X
%region ap-south-1
%iam_role arn:aws:iam::603761645750:role/glue_role_local

Setting Glue version to: 4.0
Previous number of workers: 2
Setting new number of workers to: 2
Current idle_timeout is 10 minutes.
idle_timeout has been set to 10 minutes.
Previous worker type: G.2X
Setting new worker type to: G.2X
Previous region: ap-south-1
Setting new region to: ap-south-1
Region is set to: ap-south-1
Current iam_role is arn:aws:iam::603761645750:role/glue_role_local
iam_role has been set to arn:aws:iam::603761645750:role/glue_role_local.


In [1]:
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.context import SparkContext

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

print(f" Spark Version: {spark.version}")
print(f" Spark Master : {spark.sparkContext.master}")
print(" Glue Context Ready")

Trying to create a Glue session for the kernel.
Session Type: etl
Worker Type: G.2X
Number of Workers: 2
Idle Timeout: 10
Session ID: 561108c0-0f61-4369-9b68-50e11551d2bf
Applying the following default arguments:
--glue_kernel_version 1.0.9
--enable-glue-datacatalog true
Waiting for session 561108c0-0f61-4369-9b68-50e11551d2bf to get into ready status...
Session 561108c0-0f61-4369-9b68-50e11551d2bf has been created.
 Spark Version: 3.3.0-amzn-1
 Spark Master : jes
 Glue Context Ready


In [ ]:
dyf = glueContext.create_dynamic_frame.from_options(
    connection_type="s3",
    format="csv",
    connection_options={
        "paths": ["s3://aws-cli-23/raw/titanic.csv"],
        "withHeader": True
    },
    transformation_ctx="https://aws-cli-23.s3.ap-south-1.amazonaws.com/raw/titanic.csv"
)

print(f" Records: {dyf.count()}")
dyf.printSchema()

 Records: 0
root


In [161]:
from awsglue.transforms import ResolveChoice

dyf_resolved = ResolveChoice.apply(
    frame=dyf,
    choice="make_cols",
    transformation_ctx="resolved"
)

df = dyf_resolved.toDF()
df.show()

++
||
++
++

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.


In [270]:
from awsglue.dynamicframe import DynamicFrame

dyf_output = DynamicFrame.fromDF(df, glueContext, "output_dyf")

glueContext.write_dynamic_frame.from_options(
    frame=dyf_output,
    connection_type="s3",
    format="parquet",
    connection_options={"path": "s3://aws-cli-23/clean/"},
    transformation_ctx="output"
)


print(" Data Written successfully")

AnalysisException: 
Datasource does not support writing empty or nested empty schemas.
Please make sure the data schema has at least one or more column(s).
       


In [280]:
%stop_session

Stopping session: 561108c0-0f61-4369-9b68-50e11551d2bf
Stopped session.
